In [1]:
from pyspark.sql import SparkSession

In [2]:
#spark session is the entry point to the cluster

spark = SparkSession.\
                    builder.\
                    master("spark://spark-master:7077").\
                    appName("rdd_demo").\
                    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/17 04:52:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
orders_rdd = spark.sparkContext.textFile("/data/orders_1gb.csv")

In [5]:
orders_rdd.take(10)

['order_id,customer_id,product_id,price,order_date,order_status,state,quantity',
 '1,939829,85,117.77,2008-03-31,DELIVERED,Himachal Pradesh,1',
 '2,980708,27,198.37,2015-04-30,SHIPPED,Delhi,2',
 '3,772335,22,183.63,2017-01-08,CANCELLED,Karnataka,3',
 '4,105526,75,169.02,2007-07-04,DELIVERED,Delhi,2',
 '5,676233,29,107.84,2003-03-10,CANCELLED,Haryana,1',
 '6,304518,59,163.12,2014-07-21,SHIPPED,Telangana,1',
 '7,985229,27,189.66,2003-10-21,DELIVERED,Delhi,1',
 '8,513955,19,121.61,2021-01-25,RETURNED,Delhi,2',
 '9,765941,19,118.81,2002-02-28,PLACED,Odisha,2']

In [6]:
#order_id, customer_id, product_id, price, order_date, order_status, state, quantity


In [7]:
orders_rdd.getNumPartitions()

34

In [8]:
#map ,reduce

def square_no(no):
    return no**2
square_no(5)


25

In [9]:
g=lambda x:x**2
g(5)

25

In [10]:
#map
x=list(map(lambda x:x**2,[10,5]))
x

[100, 25]

In [11]:
#reduce
from functools import reduce
x=reduce(lambda x,y:x*y,[1,2,3,4,5])
x

120

In [12]:
#split
orders_row='1,tech,200,20250117'
lst=orders_row.split(',')
lst

['1', 'tech', '200', '20250117']

# I want top 5 state with highest sales for delivered orders

In [13]:
#select state,sum(quantity * price) as total_sales from orders where order_status='delivered' group by states order by total_sales desc limit 5

# select state,total_sales from(
#select state,sum(quantity * price) as total_sales ,
#dense_rank() over(order by sum(quantity * price) desc) as dn_rnk 
#from orders where status='delivered' group b y state) tbl
#where dn_rnk=5

In [14]:
orders_rdd = spark.sparkContext.textFile("/data/orders_1gb.csv")
orders_rdd_split= orders_rdd.map(lambda x:x.split(','))
first = orders_rdd_split.first()
orders_rdd_split = orders_rdd_split.filter(lambda x:x!=first)
# filter based on delivery status
orders_rdd_filter = orders_rdd_split.filter(lambda x:x[5].upper()=='DELIVERED')
#get the state and price*quantity column
orders_rdd_map= orders_rdd_filter.map( lambda x:(x[6],float(x[3])*float(x[7])) )
# sum up the state values
orders_final=orders_rdd_map.reduceByKey(lambda x,y:x+y)
#top 5
orders_final.sortBy(lambda  x:x[1],ascending=False).take(5)

[('Haryana', 68589091.73000003),
 ('Rajasthan', 68544489.83999999),
 ('Assam', 68465525.58000003),
 ('Jharkhand', 68322156.19),
 ('West Bengal', 68298719.69999997)]

In [15]:
orders_rdd = spark.sparkContext.textFile("/data/orders_1gb.csv")

orders_rdd.take(10)

['order_id,customer_id,product_id,price,order_date,order_status,state,quantity',
 '1,939829,85,117.77,2008-03-31,DELIVERED,Himachal Pradesh,1',
 '2,980708,27,198.37,2015-04-30,SHIPPED,Delhi,2',
 '3,772335,22,183.63,2017-01-08,CANCELLED,Karnataka,3',
 '4,105526,75,169.02,2007-07-04,DELIVERED,Delhi,2',
 '5,676233,29,107.84,2003-03-10,CANCELLED,Haryana,1',
 '6,304518,59,163.12,2014-07-21,SHIPPED,Telangana,1',
 '7,985229,27,189.66,2003-10-21,DELIVERED,Delhi,1',
 '8,513955,19,121.61,2021-01-25,RETURNED,Delhi,2',
 '9,765941,19,118.81,2002-02-28,PLACED,Odisha,2']

In [16]:
orders_rdd_split= orders_rdd.map(lambda x:x.split(','))

orders_rdd_split.take(10)

[['order_id',
  'customer_id',
  'product_id',
  'price',
  'order_date',
  'order_status',
  'state',
  'quantity'],
 ['1',
  '939829',
  '85',
  '117.77',
  '2008-03-31',
  'DELIVERED',
  'Himachal Pradesh',
  '1'],
 ['2', '980708', '27', '198.37', '2015-04-30', 'SHIPPED', 'Delhi', '2'],
 ['3', '772335', '22', '183.63', '2017-01-08', 'CANCELLED', 'Karnataka', '3'],
 ['4', '105526', '75', '169.02', '2007-07-04', 'DELIVERED', 'Delhi', '2'],
 ['5', '676233', '29', '107.84', '2003-03-10', 'CANCELLED', 'Haryana', '1'],
 ['6', '304518', '59', '163.12', '2014-07-21', 'SHIPPED', 'Telangana', '1'],
 ['7', '985229', '27', '189.66', '2003-10-21', 'DELIVERED', 'Delhi', '1'],
 ['8', '513955', '19', '121.61', '2021-01-25', 'RETURNED', 'Delhi', '2'],
 ['9', '765941', '19', '118.81', '2002-02-28', 'PLACED', 'Odisha', '2']]

In [17]:
first = orders_rdd_split.first()

first

['order_id',
 'customer_id',
 'product_id',
 'price',
 'order_date',
 'order_status',
 'state',
 'quantity']

In [18]:
orders_rdd_split = orders_rdd_split.filter(lambda x:x!=first)

orders_rdd_split.take(10)

[['1',
  '939829',
  '85',
  '117.77',
  '2008-03-31',
  'DELIVERED',
  'Himachal Pradesh',
  '1'],
 ['2', '980708', '27', '198.37', '2015-04-30', 'SHIPPED', 'Delhi', '2'],
 ['3', '772335', '22', '183.63', '2017-01-08', 'CANCELLED', 'Karnataka', '3'],
 ['4', '105526', '75', '169.02', '2007-07-04', 'DELIVERED', 'Delhi', '2'],
 ['5', '676233', '29', '107.84', '2003-03-10', 'CANCELLED', 'Haryana', '1'],
 ['6', '304518', '59', '163.12', '2014-07-21', 'SHIPPED', 'Telangana', '1'],
 ['7', '985229', '27', '189.66', '2003-10-21', 'DELIVERED', 'Delhi', '1'],
 ['8', '513955', '19', '121.61', '2021-01-25', 'RETURNED', 'Delhi', '2'],
 ['9', '765941', '19', '118.81', '2002-02-28', 'PLACED', 'Odisha', '2'],
 ['10', '883551', '28', '190.5', '2016-02-13', 'SHIPPED', 'Karnataka', '3']]

In [19]:
# filter based on delivery status
orders_rdd_filter = orders_rdd_split.filter(lambda x:x[5].upper()=='DELIVERED')
orders_rdd_filter.take(5)

[['1',
  '939829',
  '85',
  '117.77',
  '2008-03-31',
  'DELIVERED',
  'Himachal Pradesh',
  '1'],
 ['4', '105526', '75', '169.02', '2007-07-04', 'DELIVERED', 'Delhi', '2'],
 ['7', '985229', '27', '189.66', '2003-10-21', 'DELIVERED', 'Delhi', '1'],
 ['14',
  '715881',
  '63',
  '144.04',
  '2017-07-20',
  'DELIVERED',
  'West Bengal',
  '2'],
 ['16', '559869', '90', '160.48', '2004-07-08', 'DELIVERED', 'Bihar', '1']]

In [20]:
#get the state and price*quantity column
orders_rdd_map= orders_rdd_filter.map( lambda x:(x[6],float(x[3])*float(x[7])) )
orders_rdd_map.take(10)

[('Himachal Pradesh', 117.77),
 ('Delhi', 338.04),
 ('Delhi', 189.66),
 ('West Bengal', 288.08),
 ('Bihar', 160.48),
 ('Tamil Nadu', 372.51),
 ('Uttar Pradesh', 167.03),
 ('Telangana', 348.68),
 ('Telangana', 345.8),
 ('West Bengal', 295.84)]

In [21]:
# sum up the state values
orders_final=orders_rdd_map.reduceByKey(lambda x,y:x+y)
orders_final.take(40)

[('Odisha', 68206019.38999997),
 ('Punjab', 68290585.64999999),
 ('West Bengal', 68298719.69999996),
 ('Tamil Nadu', 68138245.36),
 ('Delhi', 67875929.47999999),
 ('Rajasthan', 68544489.83999999),
 ('Gujarat', 68010018.50999998),
 ('Uttarakhand', 68190910.53999998),
 ('Bihar', 68201651.53999995),
 ('Uttar Pradesh', 68102579.22000001),
 ('Goa', 68133999.07000004),
 ('Telangana', 68177926.52000004),
 ('Himachal Pradesh', 68120169.48999998),
 ('Karnataka', 68168289.64),
 ('Jharkhand', 68322156.19),
 ('Kerala', 68129993.67999999),
 ('Assam', 68465525.58000003),
 ('Maharashtra', 68150969.74999999),
 ('Andhra Pradesh', 68136431.77),
 ('Madhya Pradesh', 68206519.30999999),
 ('Chhattisgarh', 67909017.41),
 ('Haryana', 68589091.73000005)]

In [22]:
#top 5
orders_final.sortBy(lambda  x:x[1],ascending=False).take(5)

[('Haryana', 68589091.73000005),
 ('Rajasthan', 68544489.83999999),
 ('Assam', 68465525.58000003),
 ('Jharkhand', 68322156.19),
 ('West Bengal', 68298719.69999997)]

# parallelize

In [23]:
data=[1,2,3,4,5,6,7,8,9,10,11,12,13,14]

rdd = spark.sparkContext.parallelize(data)

rdd.collect() #collect will give all the data

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [24]:
rdd.getNumPartitions()

4

In [25]:
rdd.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12, 13, 14]]

# Dataframe API

In [26]:
spark

In [27]:
df = spark.read.format('csv')\
            .option('header','true')\
                .option('inferSchema','true')\
                .option('samplingRatio',0.1)\
                .load('/data/orders_1gb.csv')

In [28]:
df.show(5)

+--------+-----------+----------+------+----------+------------+----------------+--------+
|order_id|customer_id|product_id| price|order_date|order_status|           state|quantity|
+--------+-----------+----------+------+----------+------------+----------------+--------+
|       1|     939829|        85|117.77|2008-03-31|   DELIVERED|Himachal Pradesh|       1|
|       2|     980708|        27|198.37|2015-04-30|     SHIPPED|           Delhi|       2|
|       3|     772335|        22|183.63|2017-01-08|   CANCELLED|       Karnataka|       3|
|       4|     105526|        75|169.02|2007-07-04|   DELIVERED|           Delhi|       2|
|       5|     676233|        29|107.84|2003-03-10|   CANCELLED|         Haryana|       1|
+--------+-----------+----------+------+----------+------------+----------------+--------+
only showing top 5 rows



In [29]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_status: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = true)



In [30]:
df.createOrReplaceTempView('orders')

In [31]:
df_sql= spark.sql("select * from orders where order_status='PLACED' ")
df_sql.show(5)


+--------+-----------+----------+------+----------+------------+--------------+--------+
|order_id|customer_id|product_id| price|order_date|order_status|         state|quantity|
+--------+-----------+----------+------+----------+------------+--------------+--------+
|       9|     765941|        19|118.81|2002-02-28|      PLACED|        Odisha|       2|
|      12|     977615|        10|196.17|2000-08-09|      PLACED|         Assam|       1|
|      15|     601312|        30|127.11|2022-12-05|      PLACED|   Maharashtra|       2|
|      17|     990529|        83|127.41|2000-04-23|      PLACED|Andhra Pradesh|       2|
|      18|     807015|        90|161.84|2001-04-13|      PLACED|         Bihar|       2|
+--------+-----------+----------+------+----------+------------+--------------+--------+
only showing top 5 rows



In [33]:
df_sql.write.mode('overwrite').format('csv').save('/data/orders/')

In [ ]:
#spark.stop()